In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
from langgraph.graph.message import add_messages

class ChatState(TypedDict):

    messages: Annotated[list[BaseMessage], add_messages]

In [ ]:
llm = ChatOpenAI()


def chat_node(state: ChatState):

    # take user query from state
    messages = state['messages']

    # send to llm
    response = llm.invoke(messages)

    # response store state
    return {'messages': [response]}

In [ ]:
#architecture of graph 

graph = StateGraph(ChatState)

# add nodes
graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

# instead of worklow saying chatbot as chatbot is also a workflow but simple 
checkpointer = MemorySaver()    # checkpointer for persistence
chatbot = graph.compile(checkpointer=checkpointer)

In [ ]:
chatbot   

In [ ]:
# doesnot allow for continuous chatting.

initial_state = {
    'messages': [HumanMessage(content='What is the capital of india')]
}

# if chatbot.invoke(initial_state)  
# gives both human and ai message appenended 

chatbot.invoke(initial_state)['messages'][-1].content  

In [ ]:
# allow for continuous chatting.

# unique id  to identify or classify session or user
thread_id = '1'     


while True:

user_message = input('Type here: ')
print('User:', user_message)

if user_message.strip().lower() in ['exit', 'quit', 'bye']:

break

config = {
    "configurable": {
        "thread_id": thread_id
    }
}

response = chatbot. invoke({'messages': [HumanMessage(content=user_message)]},config=config)

print('AI:', response['messages'][-1].content)

In [ ]:
# show workflow final state  values store   
# if we save in db it can be aceess anytime in future

state = chatbot.get_state(config=config)

# simply   chatbot.get_state(config)

In [ ]:
# show workflow start aagadi ko  pani, intermediate and final state  values store   


list(chatbot.get_state_history(config))